In [1]:
import numpy as np
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from gtts import gTTS
import os
import time
import pygame

pygame 2.5.0 (SDL 2.28.0, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
model = keras.models.load_model("D:/College/PKMKC/Sound Degla/Code/by_ade/model/data_huruf_abjad_3.h5")

In [3]:
background = None
accumulated_weight = 0.5

ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350

In [4]:
# By Syahrul
# background = None
# accumulated_weight = 0.5

# ROI_top = 300
# ROI_bottom = 650
# ROI_right = 900
# ROI_left = 1250

In [5]:
def cal_accum_avg(frame, accumulated_weight):

    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None

    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [6]:
def segment_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)

    
    _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    
    #Fetching contours in the frame (These contours can be of hand or any other object in foreground) ...
    contours, _ = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     image, contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If length of contours list = 0, means we didn't get any contours...
    if len(contours) == 0:
        return None
    else:
        # The largest external contour should be the hand 
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        # Returning the hand segment(max contour) and the thresholded image of hand...
        return (thresholded, hand_segment_max_cont)

In [7]:
# Apabila ada kata yang sama dengan kata pada index sebelumnya maka hapus kata tersebut
def clean_result(predictions):
    result_output = [predictions[0]]            # Initialize result_output with the first word
    for i in range(1, len(predictions)):
        if predictions[i] != predictions[i - 1]:
            result_output.append(predictions[i])    
    result_string = ' '.join(result_output)     # Mengubah list output menjadi string
    return result_string

In [8]:
def text_to_speech(text, filename):
    pygame.mixer.init()
    
    # Membuat objek gTTS dengan teks yang diberikan
    tts = gTTS(text=text, lang='en')

    # Simpan sebagai file suara
    tts.save(filename)

    # Putar file suara
    # os.system(f"start {filename}")
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()
    
    while pygame.mixer.music.get_busy():
        time.sleep(1)  # Tunda selama 1 detik
    pygame.mixer.quit()
    os.remove(filename)

# # Contoh penggunaan
# text = "Hello, how are you today?"
# filename = "output.mp3"
# text_to_speech(text, filename)

In [9]:
cam = cv2.VideoCapture(0)
filename = "ocdutput.mp3"

In [10]:
# cam = cv2.VideoCapture(0)
num_frames =0
# word_dict = {0:'Satu',1:'Dua',2:'Tiga',3:'Empat',4:'Lima',5:'Enam',6:'Tujuh',7:'Delapan',8:'Sembilan',9:'Sepuluh'}
# word_dict = {0:'Satu', 1:'Dua',2:'Tiga',3:'Empat',4:'Lima',5:'Enam',6:'Tujuh',7:'Delapan',8:'Sembilan',9:'Sepuluh', 10:'a', 11:'b', 12:'c', 13:'d'}
word_dict = {0:'a', 1:'b',2:'c',3:'d',4:'e',5:'f', 6:'g', 7:'h', 8:'i', 9:'k', 10:'l', 11:'o', 12:'p'}
predictions = []
while True:
    ret, frame = cam.read()

    # filpping the frame to prevent inverted image of captured frame...
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    # ROI from the frame
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]

    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)


    if num_frames < 70:
        cal_accum_avg(gray_frame, accumulated_weight)
        cv2.putText(frame_copy, "FETCHING BACKGROUND...PLEASE WAIT", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    else: 
        # segmenting the hand region
        hand = segment_hand(gray_frame)
        

        # Checking if we are able to detect the hand...
        if hand is not None:
            
            thresholded, hand_segment = hand

            # Drawing contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right, ROI_top)], -1, (255, 0, 0),1)
            
            cv2.imshow("Thesholded Hand Image", thresholded)
            
            thresholded = cv2.resize(thresholded, (64, 64))
            thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2RGB)
            thresholded = np.reshape(thresholded, (1,thresholded.shape[0],thresholded.shape[1],3))
            
            pred = model.predict(thresholded)
            pred_word = word_dict[np.argmax(pred)]
            cv2.putText(frame_copy, pred_word, (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            predictions.append(pred_word)
            if len(clean_result(predictions)) >= 10:
                text_to_speech(clean_result(predictions), filename)
                print(f'\n--------{len(clean_result(predictions))}')
                print(clean_result(predictions))
                predictions = []
                
            
            
    # Draw ROI on frame_copy
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (255,128,0), 3)

    # incrementing the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.putText(frame_copy, "DataFlair hand sign recognition_ _ _", (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Sign Detection", frame_copy)


    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break
        
# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 92ms/step

--------11
a b c b c a
1/1 [==============================] - 0s 75ms/step

--------11
a c a b c a
1/1 [==============================] - 0s 54ms/step

--------11
a b c b a c
1/1 [==============================] - 0s 68ms/step

--------11
a b c a c a
1/1 [==============================] - 0s 87ms/step

--------11
a b c b c b
1/1 [==============================] - 0s 63ms/step

--------11
b c b c b c
1/1 [==============================] - 0s 54ms/step

--------11
c b a c a b
1/1 [==============================] - 0s 52ms/step


In [11]:
# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()